# Imports

In [20]:
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', None)
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score

# Data

In [8]:
# Read in train and test data.
# Fill in TODO with path to csv files.
train_df = pd.read_csv('TODO')
test_df = pd.read_csv('TODO')

# Training data.
X_train = train_df['review_text']
y_train = train_df['label']

# Testing data.
X_test = test_df['review_text']

# Baseline Model

In [12]:
# Build pipeline to vectorize text and train classifier.
model_pipe = Pipeline([('tfidf', TfidfVectorizer()), ('knn', KNeighborsClassifier())])
# Train pipeline
model_pipe.fit(X_train, y_train)
# Get predictions for submission.
preds = model_pipe.predict(X_test)

# Get out-of-sample predicted probabilities

In [23]:
# Get out-of-sample predicted probabilities for training data to pass into cleanlab.
pred_probs = cross_val_predict(model_pipe, X_train, y_train, method='predict_proba')
# Cross validation accuracy on training data.
cross_val_preds = np.argmax(pred_probs, axis=1)
# Adjust for labels being 1-5
cross_val_preds += 1
acc = accuracy_score(y_train, cross_val_preds)

# Find Label Issues

In [24]:
from cleanlab.filter import find_label_issues

# Helper to convert given labels to 0-indexed (necessary for cleanlab and various ML processes).
def labels_to_zeroidx(labels):
    unique_labels = set(labels)
    label2idx = dict(zip(unique_labels, range(len(unique_labels))))
    return [label2idx[label] for label in labels]

# Compute label issues and get their indices.
# Note: all of these are not necessarily label errors. You'll need to improve model performance to get better 
# label preditions.
labels = labels_to_zeroidx(y_train)
issues_idx = find_label_issues(labels, pred_probs, return_indices_ranked_by='self_confidence')

# Show the top 5.
train_df.iloc[issues_idx][:5]

review_id  \
12641      65112   
5440       80317   
5442       14788   
5443       14789   
5460       80394   

                                                                                                                                                                  review_text  \
12641                                                                             to big, very impersonal. wish we had subscribed to regular print version of Readers digest.   
5440                                                                                        Ordered it for an 11 year old horse lover and she wasn't really excited about it.   
5442                                                                                                                                                Not the mag it used to be   
5443                                                           Great content. If you use the harbor freight coupon in the back of the magazine it pays for itself in no time.   
5460   The information is good, but for almost $300 a year, I don't expect to get screen blinding ads or click bait for "Hot Women of the '70's, what do they look like now?"   

       label  
12641      2  
5440       3  
5442       1  
5443       4  
5460       2

# Train a Better Model!

In [25]:
# Drop all issues from training data.
X_train_clean = X_train.drop(issues_idx)
y_train_clean = y_train.drop(issues_idx)

# Re-train model on data with issues removed.
model_pipe = Pipeline([('tfidf', TfidfVectorizer()), ('knn', KNeighborsClassifier())])
model_pipe.fit(X_train_clean, y_train_clean)
# Make predictions for submission.
preds = model_pipe.predict(X_test)

# Food for Thought

- The (very basic) KNN classifier used to find the `pred_probs` is not very accurate - improving this classifier might improve quality of issues found, too.
- Here we just dropped all of the issues, what else could you do?
- Other useful modules:
    - [cleanlab.outlier](https://github.com/cleanlab/cleanlab/blob/master/cleanlab/outlier.py)
    - [cleanlab.classification](https://github.com/cleanlab/cleanlab/blob/master/cleanlab/classification.py)
- Resources:
    - [GitHub](https://github.com/cleanlab/cleanlab)
    - [Docs](https://docs.cleanlab.ai/stable/index.html)
    
